In [1]:
#Source: https://www.kaggle.com/datasets/stefanoleone992/fifa-22-complete-player-dataset?resource=download
#Only run on linux 
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import autosklearn
pd.set_option('display.max_columns', 500)

In [2]:
# Data loading and Cleaning

players_22=pd.read_csv("players_22.csv",low_memory=False)
players_22=players_22.drop(columns=['release_clause_eur','club_contract_valid_until','club_joined','nation_jersey_number','nation_position','nation_team_id','club_loaned_from','nation_jersey_number','body_type','real_face','player_tags','player_traits','long_name','sofifa_id','player_url','player_face_url', 'club_logo_url','club_flag_url', 'nation_logo_url', 'nation_flag_url'])
for i in range(0,19239,1):
    for j in range(64,91,1):
        original=players_22.iloc[i,j]
        players_22.iloc[i,j]=eval(original)
best_position=players_22.iloc[:,64:91].astype('float64').idxmax(axis=1,skipna=True).to_frame(name="best_position")
players_22=pd.concat([players_22,best_position],axis=1)

le=LabelEncoder()
position_label=le.fit_transform(players_22["best_position"])
df=pd.DataFrame(position_label,columns=["position_label"])
players_22=pd.concat([players_22,df],axis=1)
print(players_22)

               short_name player_positions  overall  potential    value_eur  \
0                L. Messi       RW, ST, CF       93         93   78000000.0   
1          R. Lewandowski               ST       92         92  119500000.0   
2       Cristiano Ronaldo           ST, LW       91         91   45000000.0   
3               Neymar Jr          LW, CAM       91         91  129000000.0   
4            K. De Bruyne          CM, CAM       91         91  125500000.0   
...                   ...              ...      ...        ...          ...   
19234           Song Defu              CDM       47         52      70000.0   
19235           C. Porter               CM       47         59     110000.0   
19236            N. Logue               CM       47         55     100000.0   
19237           L. Rudden               ST       47         60     110000.0   
19238  E. Lalchhanchhuaha              CAM       47         60     110000.0   

       wage_eur  age         dob  height_cm  weight

In [9]:
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
#positioin classification
index=['pace', 'shooting', 'passing', 'dribbling', 'physic',
       'attacking_crossing', 'attacking_finishing', 'skill_dribbling',
       'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
       'power_stamina', 'mentality_positioning', 'goalkeeping_diving',
       'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']
y=players_22['position_label']
x=players_22[index].fillna(0)
scaler = StandardScaler()
x_standard=pd.DataFrame(scaler.fit_transform(x),columns=x.columns)
display(x_standard)
X_train,X_test,y_train,y_test=train_test_split(x_standard,y,train_size=0.8,random_state=2023,shuffle=True)
automlclassifierV1=AutoSklearn2Classifier(time_left_for_this_task=3600,memory_limit=5500,n_jobs=-1)
automlclassifierV1.fit(X_train,y_train)
print(automlclassifierV1.score(X_test,y_test))


,pace,shooting,passing,dribbling,physic,attacking_crossing,attacking_finishing,skill_dribbling,skill_ball_control,movement_acceleration,movement_sprint_speed,power_stamina,mentality_positioning,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,1.024453,2.153464,1.968245,1.819019,0.329387,1.964190,2.490076,2.147538,2.252136,1.737133,1.021387,0.552196,2.174690,-0.592145,-0.308358,-0.063713,-0.130678,-0.474818,-0.336925
1,0.729901,2.153464,1.378341,1.403208,1.090178,1.187887,2.490076,1.561936,1.772039,0.814077,0.954565,0.799952,2.276621,-0.080012,-0.605286,-0.244828,-0.482391,-0.362988,-0.336925
2,1.108610,2.248215,1.427500,1.495610,0.776911,2.075090,2.490076,1.721646,1.772039,1.341538,1.555967,0.861892,2.276621,-0.535241,-0.308358,-0.063713,-0.130678,-0.307073,-0.336925
3,1.276925,1.727087,1.722452,1.772817,0.239882,1.964190,1.881572,2.094301,2.192124,1.868998,1.622789,1.109649,1.817931,-0.421434,-0.427130,-0.063713,-0.072059,-0.307073,-0.336925
4,0.645744,1.869213,2.066563,1.495610,0.911169,2.463242,1.830864,1.721646,1.952075,0.748144,0.754098,1.605162,1.919862,-0.080012,-0.189587,-0.667428,-0.365153,-0.195244,-0.336925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,-0.111674,-0.546927,-0.243895,-0.352440,-0.386652,-0.198369,-0.704568,-0.567524,-0.568436,-0.570508,-0.315061,-0.748528,-0.628416,-0.592145,-0.367744,-0.667428,-0.072059,-0.195244,-0.336925
19235,-0.069595,-0.357426,-0.047260,-0.444842,-0.297147,0.245233,-0.653859,-0.780470,-0.988521,-0.306777,-0.448706,-0.005257,0.034136,-0.307627,-0.248973,-0.607056,-0.482391,-0.362988,-0.336925
19236,-0.027517,-0.452177,-0.293054,-0.306238,-0.252394,-0.586520,-0.704568,-0.461051,-0.568436,-0.306777,-0.315061,-0.438832,-0.271657,-0.478338,-0.605286,-0.546685,-0.365153,-0.586647,-0.336925
19237,0.309114,-0.025799,-0.735482,-0.352440,-0.699919,-1.141023,0.157479,-0.727234,-0.808484,0.286616,0.152696,-0.810467,-0.169726,-0.535241,-0.367744,-0.546685,-0.130678,-0.083414,-0.336925


/home/ubuntu/.local/lib/python3.8/site-packages/autosklearn/experimental/selector.py:26: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, series in prediction.iteritems():


[WARNING] [2023-05-12 01:09:32,646:Client-AutoML(1):959a54bd-f01e-11ed-bcc8-72fb63aef1f8] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (3599.379508)
[WARNING] [2023-05-12 01:09:32,646:Client-AutoML(1):959a54bd-f01e-11ed-bcc8-72fb63aef1f8] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
0.757016632016632


In [11]:
automlclassifierV1.leaderboard(detailed = True, ensemble_only=True)
from joblib import dump, load

# 匯出模型
dump(automlclassifierV1, 'model.joblib') 

['model.joblib']

In [12]:
print(automlclassifierV1.leaderboard(detailed = True, ensemble_only=True))

          rank  ensemble_weight type      cost    duration  config_id  \
model_id                                                                
631          1             0.10  mlp  0.231824  484.772128        630   
595          2             0.04  mlp  0.233058  158.969112        594   
627          3             0.02  mlp  0.233643  155.646739        626   
488          4             0.20  mlp  0.233643  113.347836        487   
273          5             0.02  mlp  0.233708  140.660575        272   
322          6             0.04  mlp  0.233708  112.850147        321   
674          7             0.04  mlp  0.233773  161.616814        673   
309          8             0.02  mlp  0.233773  119.140565        308   
416          9             0.04  mlp  0.233903  112.024382        415   
773         10             0.04  mlp  0.234033  216.006617        772   
400         11             0.04  mlp  0.234163  112.146634        399   
700         12             0.02  mlp  0.234163  174

In [4]:
#wage regression
from autosklearn.regression import AutoSklearnRegressor
index=[ 'overall', 'shooting', 'passing', 'dribbling',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_reactions', 'power_shot_power', 'power_long_shots',
       'mentality_aggression', 'mentality_positioning', 'mentality_vision',
       'mentality_penalties', 'mentality_composure']
y=players_22['wage_eur'].fillna(0)
x=players_22[index]
x['shooting']=x['shooting'].fillna(x['shooting'].mean())
x['passing']=x['passing'].fillna(x['passing'].mean())
x['dribbling']=x['dribbling'].fillna(x['dribbling'].mean())

X_train,X_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=2023,shuffle=True)
autoR=AutoSklearnRegressor(time_left_for_this_task=120*60,memory_limit=5500,n_jobs=-1)
autoR.fit(X_train,y_train)
print(autoR.score(X_test,y_test))

/tmp/ipykernel_55382/75317589.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['shooting']=x['shooting'].fillna(x['shooting'].mean())
/tmp/ipykernel_55382/75317589.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['passing']=x['passing'].fillna(x['passing'].mean())
/tmp/ipykernel_55382/75317589.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

[WARNING] [2023-05-13 01:20:48,795:Client-AutoML(1):5624eab7-f0e9-11ed-9856-0e973809ea74] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (7199.477437)
[WARNING] [2023-05-13 01:20:48,796:Client-AutoML(1):5624eab7-f0e9-11ed-9856-0e973809ea74] Capping the per_run_time_limit to 3599.0 to have time for a least 2 models in each process.


/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
/home/ubuntu/.local/lib/python3.8/site-packag

[ERROR] [2023-05-13 02:11:28,268:Client-AutoML(1):5624eab7-f0e9-11ed-9856-0e973809ea74] [Errno 28] No space left on device: '/tmp/auto-sklearn_tmp_562499b4-f0e9-11ed-9856-0e973809ea74/smac3-output/run_1/stats.json'
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/smac/facade/smac_ac_facade.py", line 720, in optimize
    incumbent = self.solver.run()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 316, in run
    self._incorporate_run_results(run_info, result, time_left)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 563, in _incorporate_run_results
    self.save()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 571, in save
    self.stats.save()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/smac/stats/stats.py", line 78, in save
    with open(path, 'w') as fh:
OSError: [Errno 28] No space left on device: '/tmp/auto-sklear

OSError: [Errno 28] No space left on device: '/tmp/auto-sklearn_tmp_562499b4-f0e9-11ed-9856-0e973809ea74/smac3-output/run_1/stats.json'